# Starter Notebook: GEO AI  Hackathon!


Welcome! This starter notebook is designed to get you started on this challenge, where you will be using time-series Sentinel-2 multi-spectral data to classify crops. We will take a look at the data, create a model and then use that to make our first submission. After that we will briefly look at some ways to improve. Let's get started.

https://zindi.africa/competitions/geoai-hack-2022


---


## Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import log_loss , accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter('ignore')

## Mean

In [2]:
train = pd.read_csv('Train.csv')

foo = train.iloc[:, :2]

trainx = train.iloc[:, 2:].filter(like='_mean')

train = pd.concat([foo, trainx], axis=1)

train.filter(regex='timestep _')

l = []
for i in range(10, len(train.columns), 10):
    l.append(train.columns[i])
    l.append(train.columns[i+1])
    
 

train.drop(l, axis=1, inplace=True)

train.shape

train.head()

,ID,Target,timestep1_B02_mean,timestep1_B03_mean,timestep1_B04_mean,timestep1_B05_mean,timestep1_B06_mean,timestep1_B07_mean,timestep1_B08_mean,timestep1_B8A_mean,...,timestep23_B08_mean,timestep23_B8A_mean,timestep24_B02_mean,timestep24_B03_mean,timestep24_B04_mean,timestep24_B05_mean,timestep24_B06_mean,timestep24_B07_mean,timestep24_B08_mean,timestep24_B8A_mean
0,6fba03cb,olive+cereals,456.918182,799.361364,825.761364,1420.606818,2478.922727,2882.245455,3084.704545,1989.954545,...,2809.536364,2447.077273,590.368182,1111.515909,1400.540909,1715.302273,2162.463636,2312.918182,2436.863636,2639.679545
1,7ea60a74,plowing_and_sowing,1087.130208,1653.770833,2194.458333,2447.984375,2588.328125,2725.609375,2777.385417,3698.088542,...,3307.052083,2202.869792,575.468750,1126.234375,1338.671875,1768.291667,2585.895833,2765.562500,2878.385417,2618.859375
2,64d18595,olive,917.369369,1383.882883,1742.765766,2012.234234,2388.657658,2592.783784,2644.738739,3111.837838,...,3697.405405,2348.423423,360.063063,895.252252,784.621622,1520.954955,3140.477477,3480.045045,3694.090090,2387.882883
3,119c8ec4,arable_soil,1013.775120,1551.863636,2073.009569,2371.222488,2666.064593,2843.196172,2930.330144,3802.370813,...,2573.497608,3388.679426,827.315789,1422.665072,1855.571770,2271.423445,2721.198565,2860.514354,3074.009569,3484.294258
4,d884d98c,olive,801.352113,1243.741784,1557.410798,1827.133803,2248.718310,2433.793427,2471.901408,2846.659624,...,2202.943662,2174.800469,352.042254,718.793427,959.312207,1171.706573,1586.760563,1789.600939,1904.948357,2046.302817


In [3]:
test = pd.read_csv('Test.csv')

In [4]:
foo = test.iloc[:, :1]

testx = test.iloc[:, 1:].filter(like='_mean')

test = pd.concat([foo, testx], axis=1)

test.filter(regex='timestep _')

l = []
for i in range(9, len(test.columns), 10):
    l.append(test.columns[i])
    l.append(test.columns[i+1])
    
 

test.drop(l, axis=1, inplace=True)

test.shape

(502, 193)

In [5]:
train['timestep2_B04_mean'].head()

0     748.068182
1    2179.666667
2    2144.936937
3    2316.100478
4    1830.016432
Name: timestep2_B04_mean, dtype: float64

## calculate ndvi

In [6]:
# add 24 ndvi

for i in range(1,25):
  ttt = train.filter(like='timestep'+str(i)+'_')
  train['train_ndvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B04_mean']) / (ttt['timestep'+str(i)+'_B8A_mean'] + ttt['timestep'+str(i)+'_B04_mean'])


In [7]:

for i in range(1,25):
  ttt = test.filter(like='timestep'+str(i)+'_')
  test['train_ndvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B04_mean']) / (ttt['timestep'+str(i)+'_B8A_mean'] + ttt['timestep'+str(i)+'_B04_mean'])


In [8]:
train.head()

,ID,Target,timestep1_B02_mean,timestep1_B03_mean,timestep1_B04_mean,timestep1_B05_mean,timestep1_B06_mean,timestep1_B07_mean,timestep1_B08_mean,timestep1_B8A_mean,...,train_ndvi_15,train_ndvi_16,train_ndvi_17,train_ndvi_18,train_ndvi_19,train_ndvi_20,train_ndvi_21,train_ndvi_22,train_ndvi_23,train_ndvi_24
0,6fba03cb,olive+cereals,456.918182,799.361364,825.761364,1420.606818,2478.922727,2882.245455,3084.704545,1989.954545,...,0.518206,0.646110,0.602690,0.638452,0.636711,0.502919,0.413053,0.335179,0.397782,0.306701
1,7ea60a74,plowing_and_sowing,1087.130208,1653.770833,2194.458333,2447.984375,2588.328125,2725.609375,2777.385417,3698.088542,...,0.334613,0.335088,0.378112,0.411510,0.418998,0.397873,0.421284,0.349049,0.412999,0.323481
2,64d18595,olive,917.369369,1383.882883,1742.765766,2012.234234,2388.657658,2592.783784,2644.738739,3111.837838,...,0.458716,0.449301,0.508220,0.540008,0.551040,0.543415,0.510145,0.411125,0.518961,0.505361
3,119c8ec4,arable_soil,1013.775120,1551.863636,2073.009569,2371.222488,2666.064593,2843.196172,2930.330144,3802.370813,...,0.327453,0.305213,0.325036,0.331254,0.315194,0.323119,0.326130,0.317126,0.330759,0.305012
4,d884d98c,olive,801.352113,1243.741784,1557.410798,1827.133803,2248.718310,2433.793427,2471.901408,2846.659624,...,0.307998,0.323709,0.336009,0.344739,0.329177,0.326522,0.320774,0.289737,0.299538,0.361653


## calculate evi

In [9]:
# add 24 evi
'''
for i in range(1,25):
  ttt = train.filter(like='timestep'+str(i)+'_')
  train['train_evi_'+str(i)] = \
    2.5*(ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B04_mean']) / \
    (ttt['timestep'+str(i)+'_B8A_mean'] + 6*ttt['timestep'+str(i)+'_B04_mean'] - 7.5*ttt['timestep'+str(i)+'_B02_mean']+1)
'''

"\nfor i in range(1,25):\n  ttt = train.filter(like='timestep'+str(i)+'_')\n  train['train_evi_'+str(i)] =     2.5*(ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B04_mean']) /     (ttt['timestep'+str(i)+'_B8A_mean'] + 6*ttt['timestep'+str(i)+'_B04_mean'] - 7.5*ttt['timestep'+str(i)+'_B02_mean']+1)\n"

In [10]:
'''
for i in range(1,25):
  ttt = test.filter(like='timestep'+str(i)+'_')
  test['train_evi_'+str(i)] = \
    2.5*(ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B04_mean']) / \
    (ttt['timestep'+str(i)+'_B8A_mean'] + 6*ttt['timestep'+str(i)+'_B04_mean'] - 7.5*ttt['timestep'+str(i)+'_B02_mean']+1)
'''

"\nfor i in range(1,25):\n  ttt = test.filter(like='timestep'+str(i)+'_')\n  test['train_evi_'+str(i)] =     2.5*(ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B04_mean']) /     (ttt['timestep'+str(i)+'_B8A_mean'] + 6*ttt['timestep'+str(i)+'_B04_mean'] - 7.5*ttt['timestep'+str(i)+'_B02_mean']+1)\n"

## calculate gndvi

In [11]:
# add 24 gndvi
'''
for i in range(1,25):
  ttt = train.filter(like='timestep'+str(i)+'_')
  train['train_gndvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B03_mean']) / (ttt['timestep'+str(i)+'_B8A_mean'] + ttt['timestep'+str(i)+'_B03_mean'])
'''

"\nfor i in range(1,25):\n  ttt = train.filter(like='timestep'+str(i)+'_')\n  train['train_gndvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B03_mean']) / (ttt['timestep'+str(i)+'_B8A_mean'] + ttt['timestep'+str(i)+'_B03_mean'])\n"

In [12]:
'''
for i in range(1,25):
  ttt = test.filter(like='timestep'+str(i)+'_')
  test['train_gndvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B03_mean']) / (ttt['timestep'+str(i)+'_B8A_mean'] + ttt['timestep'+str(i)+'_B03_mean'])
'''

"\nfor i in range(1,25):\n  ttt = test.filter(like='timestep'+str(i)+'_')\n  test['train_gndvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B03_mean']) / (ttt['timestep'+str(i)+'_B8A_mean'] + ttt['timestep'+str(i)+'_B03_mean'])\n"

## calculate cvi

In [13]:
# add 24 cvi
'''
for i in range(1,25):
  ttt = train.filter(like='timestep'+str(i)+'_')
  train['train_cvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] / ttt['timestep'+str(i)+'_B03_mean']) * (ttt['timestep'+str(i)+'_B04_mean'] / ttt['timestep'+str(i)+'_B03_mean'])
'''

"\nfor i in range(1,25):\n  ttt = train.filter(like='timestep'+str(i)+'_')\n  train['train_cvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] / ttt['timestep'+str(i)+'_B03_mean']) * (ttt['timestep'+str(i)+'_B04_mean'] / ttt['timestep'+str(i)+'_B03_mean'])\n"

In [14]:
'''
for i in range(1,25):
  ttt = test.filter(like='timestep'+str(i)+'_')
  test['train_cvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] / ttt['timestep'+str(i)+'_B03_mean']) * (ttt['timestep'+str(i)+'_B04_mean'] / ttt['timestep'+str(i)+'_B03_mean'])
'''

"\nfor i in range(1,25):\n  ttt = test.filter(like='timestep'+str(i)+'_')\n  test['train_cvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] / ttt['timestep'+str(i)+'_B03_mean']) * (ttt['timestep'+str(i)+'_B04_mean'] / ttt['timestep'+str(i)+'_B03_mean'])\n"

## delete B8A and B04 and B02

In [15]:
###### tra
# delete B8A
'''
to_delete_B8A_train = train.filter(like='_B8A_')

train.drop(to_delete_B8A_train, axis=1, inplace=True)

# delete B04
to_delete_B04_train = train.filter(like='_B04_')

train.drop(to_delete_B04_train, axis=1, inplace=True)


# delete B02
to_delete_B02_train = train.filter(like='_B02_')

train.drop(to_delete_B02_train, axis=1, inplace=True)
'''

"\nto_delete_B8A_train = train.filter(like='_B8A_')\n\ntrain.drop(to_delete_B8A_train, axis=1, inplace=True)\n\n# delete B04\nto_delete_B04_train = train.filter(like='_B04_')\n\ntrain.drop(to_delete_B04_train, axis=1, inplace=True)\n\n\n# delete B02\nto_delete_B02_train = train.filter(like='_B02_')\n\ntrain.drop(to_delete_B02_train, axis=1, inplace=True)\n"

In [16]:
###### test
'''
# delete B8A

to_delete_B8A_test = test.filter(like='_B8A_')

test.drop(to_delete_B8A_test, axis=1, inplace=True)

# delete B04
to_delete_B04_test = test.filter(like='_B04_')

test.drop(to_delete_B04_test, axis=1, inplace=True)

# delete B02
to_delete_B02_test = test.filter(like='_B02_')

test.drop(to_delete_B02_test, axis=1, inplace=True)
'''

"\n# delete B8A\n\nto_delete_B8A_test = test.filter(like='_B8A_')\n\ntest.drop(to_delete_B8A_test, axis=1, inplace=True)\n\n# delete B04\nto_delete_B04_test = test.filter(like='_B04_')\n\ntest.drop(to_delete_B04_test, axis=1, inplace=True)\n\n# delete B02\nto_delete_B02_test = test.filter(like='_B02_')\n\ntest.drop(to_delete_B02_test, axis=1, inplace=True)\n"

## Delete all rows except last 24

In [17]:
# delete all rows except last 24

to_delete_train = train.filter(like='timestep')
train.drop(to_delete_train, axis=1, inplace=True)

to_delete_test = test.filter(like='timestep')
test.drop(to_delete_test, axis=1, inplace=True)


## Scaling

In [18]:
train.head()

,ID,Target,train_ndvi_1,train_ndvi_2,train_ndvi_3,train_ndvi_4,train_ndvi_5,train_ndvi_6,train_ndvi_7,train_ndvi_8,...,train_ndvi_15,train_ndvi_16,train_ndvi_17,train_ndvi_18,train_ndvi_19,train_ndvi_20,train_ndvi_21,train_ndvi_22,train_ndvi_23,train_ndvi_24
0,6fba03cb,olive+cereals,0.413463,0.464631,0.520925,0.673917,0.615928,0.428587,0.424880,0.350509,...,0.518206,0.646110,0.602690,0.638452,0.636711,0.502919,0.413053,0.335179,0.397782,0.306701
1,7ea60a74,plowing_and_sowing,0.255175,0.258223,0.333200,0.394139,0.400159,0.407836,0.426992,0.385118,...,0.334613,0.335088,0.378112,0.411510,0.418998,0.397873,0.421284,0.349049,0.412999,0.323481
2,64d18595,olive,0.282015,0.261907,0.457275,0.565093,0.553390,0.550971,0.523822,0.579131,...,0.458716,0.449301,0.508220,0.540008,0.551040,0.543415,0.510145,0.411125,0.518961,0.505361
3,119c8ec4,arable_soil,0.294340,0.283934,0.316927,0.306772,0.323197,0.327781,0.333833,0.325215,...,0.327453,0.305213,0.325036,0.331254,0.315194,0.323119,0.326130,0.317126,0.330759,0.305012
4,d884d98c,olive,0.292740,0.279449,0.295647,0.333862,0.341406,0.334100,0.333767,0.323873,...,0.307998,0.323709,0.336009,0.344739,0.329177,0.326522,0.320774,0.289737,0.299538,0.361653


In [19]:

from sklearn.preprocessing import StandardScaler



scaler = StandardScaler()

# train
to_scale_train = train.drop(['ID', 'Target'], axis=1).columns
scaler.fit(train[to_scale_train])

train[to_scale_train] = scaler.transform(train[to_scale_train])

# test
to_scale_test = test.drop('ID', axis=1).columns
scaler.fit(test[to_scale_test])

test[to_scale_test] = scaler.transform(test[to_scale_test])


print(test.head())



"\n\nscaler = StandardScaler()\n\n# train\nto_scale_train = train.drop(['ID', 'Target'], axis=1).columns\nscaler.fit(train[to_scale_train])\n\ntrain[to_scale_train] = scaler.transform(train[to_scale_train])\n\n# test\nto_scale_test = test.drop('ID', axis=1).columns\nscaler.fit(test[to_scale_test])\n\ntest[to_scale_test] = scaler.transform(test[to_scale_test])\n\n\nprint(test.head())\n\n"

In [20]:
train.head()

,ID,Target,train_ndvi_1,train_ndvi_2,train_ndvi_3,train_ndvi_4,train_ndvi_5,train_ndvi_6,train_ndvi_7,train_ndvi_8,...,train_ndvi_15,train_ndvi_16,train_ndvi_17,train_ndvi_18,train_ndvi_19,train_ndvi_20,train_ndvi_21,train_ndvi_22,train_ndvi_23,train_ndvi_24
0,6fba03cb,olive+cereals,0.413463,0.464631,0.520925,0.673917,0.615928,0.428587,0.424880,0.350509,...,0.518206,0.646110,0.602690,0.638452,0.636711,0.502919,0.413053,0.335179,0.397782,0.306701
1,7ea60a74,plowing_and_sowing,0.255175,0.258223,0.333200,0.394139,0.400159,0.407836,0.426992,0.385118,...,0.334613,0.335088,0.378112,0.411510,0.418998,0.397873,0.421284,0.349049,0.412999,0.323481
2,64d18595,olive,0.282015,0.261907,0.457275,0.565093,0.553390,0.550971,0.523822,0.579131,...,0.458716,0.449301,0.508220,0.540008,0.551040,0.543415,0.510145,0.411125,0.518961,0.505361
3,119c8ec4,arable_soil,0.294340,0.283934,0.316927,0.306772,0.323197,0.327781,0.333833,0.325215,...,0.327453,0.305213,0.325036,0.331254,0.315194,0.323119,0.326130,0.317126,0.330759,0.305012
4,d884d98c,olive,0.292740,0.279449,0.295647,0.333862,0.341406,0.334100,0.333767,0.323873,...,0.307998,0.323709,0.336009,0.344739,0.329177,0.326522,0.320774,0.289737,0.299538,0.361653


In [21]:
test.head()

,ID,train_ndvi_1,train_ndvi_2,train_ndvi_3,train_ndvi_4,train_ndvi_5,train_ndvi_6,train_ndvi_7,train_ndvi_8,train_ndvi_9,...,train_ndvi_15,train_ndvi_16,train_ndvi_17,train_ndvi_18,train_ndvi_19,train_ndvi_20,train_ndvi_21,train_ndvi_22,train_ndvi_23,train_ndvi_24
0,d8da32b5,0.368313,0.371175,0.371148,0.404169,0.400864,0.392843,0.393875,0.362949,0.358921,...,0.384491,0.398766,0.394720,0.410702,0.409937,0.405672,0.386889,0.332931,0.376264,0.358241
1,670ad0fb,0.342660,0.333081,0.333788,0.351294,0.345466,0.312162,0.281990,0.287264,0.309080,...,0.336212,0.314798,0.357455,0.381305,0.343121,0.328595,0.311645,0.288753,0.303456,0.279377
2,fec40ac9,0.353936,0.341969,0.314638,0.300414,0.319950,0.377002,0.383119,0.339756,0.344342,...,0.322032,0.299869,0.320002,0.320394,0.301170,0.352460,0.364045,0.361838,0.388710,0.386024
3,4f6d4495,0.442174,0.465993,0.395177,0.395792,0.405980,0.380806,0.400669,0.367576,0.382000,...,0.428792,0.432072,0.412892,0.412241,0.393747,0.381377,0.359853,0.347697,0.371073,0.439725
4,e56d2db7,0.249619,0.242173,0.457597,0.485632,0.468038,0.387682,0.392634,0.337523,0.348073,...,0.312780,0.498693,0.485278,0.493125,0.462244,0.408974,0.384436,0.322719,0.388360,0.315873


In [22]:
# for 24 columns
'''

from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

# train
to_scale_train = train.drop(['ID', 'Target'], axis=1).columns
scaler.fit(train[to_scale_train])

train[to_scale_train] = scaler.transform(train[to_scale_train])

# test
to_scale_test = test.drop('ID', axis=1).columns

scaler.fit(test[to_scale_test])

test[to_scale_test] = scaler.transform(test[to_scale_test])



print(test.head())
'''

         ID  train_ndvi_1  train_ndvi_2  train_ndvi_3  train_ndvi_4  \
0  d8da32b5      0.319765      0.231322      0.024700      0.014195   
1  670ad0fb     -0.106655     -0.252914     -0.469677     -0.389538   
2  fec40ac9      0.080779     -0.139930     -0.723073     -0.778036   
3  4f6d4495      1.547509      1.436596      0.342662     -0.049763   
4  e56d2db7     -1.653209     -1.408496      1.168643      0.636218   

   train_ndvi_5  train_ndvi_6  train_ndvi_7  train_ndvi_8  train_ndvi_9  ...  \
0     -0.033950      0.029400      0.040636     -0.142692     -0.178751  ...   
1     -0.532930     -0.780399     -1.178829     -0.967963     -0.894188  ...   
2     -0.762758     -0.129596     -0.076592     -0.395590     -0.388026  ...   
3      0.012128     -0.091414      0.114688     -0.092243      0.152538  ...   
4      0.571103     -0.022393      0.027116     -0.419941     -0.334468  ...   

   train_ndvi_15  train_ndvi_16  train_ndvi_17  train_ndvi_18  train_ndvi_19  \
0       0.13

## Removing correlated features

In [23]:
# train

cor_matrix = train.corr().abs()


upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))


to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

print(len(to_drop))

train.drop(to_drop, axis=1, inplace=True)


# test


test.drop(to_drop, axis=1, inplace=True)
print(train.shape)
print(test.shape)


13
(1004, 13)
(502, 12)


## Variance

In [24]:
'''from sklearn.feature_selection import RFE

# Set the feature eliminator to remove 2 features on each step
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=3, step=2, verbose=1)

# Fit the model to the training data
rfe.fit(train.drop(['ID', 'Target'], axis=1), train['Target'])

# Create a mask
mask = rfe.support_

print(mask)

# Apply the mask to the feature dataset X and print the result
reduced_train = train.drop(['ID', 'Target']).loc[:, mask]


train = train.loc[:,[True,True, True, False, False, False, False, False, False, False, False,False, False ,False,
 False, False ,False ,False ,False, False, False,False ,False ,False, False, False,
 False, False ,False, False, False , True, False, False, False, False, False, False,
 False, False ,False, False, False, False ,False, False, False,  True, False, False,
 False, False, False, False, False, False, False]]

test = test.loc[:,[True, True, False, False, False, False, False, False, False, False,False, False ,False,
 False, False ,False ,False ,False, False, False,False ,False ,False, False, False,
 False, False ,False,False, False , True, False, False, False, False, False, False,
 False, False ,False, False, False, False ,False, False, False,  True, False, False,
 False, False, False, False, False, False, False]]'''

"from sklearn.feature_selection import RFE\n\n# Set the feature eliminator to remove 2 features on each step\nrfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=3, step=2, verbose=1)\n\n# Fit the model to the training data\nrfe.fit(train.drop(['ID', 'Target'], axis=1), train['Target'])\n\n# Create a mask\nmask = rfe.support_\n\nprint(mask)\n\n# Apply the mask to the feature dataset X and print the result\nreduced_train = train.drop(['ID', 'Target']).loc[:, mask]\n\n\ntrain = train.loc[:,[True,True, True, False, False, False, False, False, False, False, False,False, False ,False,\n False, False ,False ,False ,False, False, False,False ,False ,False, False, False,\n False, False ,False, False, False , True, False, False, False, False, False, False,\n False, False ,False, False, False, False ,False, False, False,  True, False, False,\n False, False, False, False, False, False, False]]\n\ntest = test.loc[:,[True, True, False, False, False, False, False, False, False, False,

In [25]:
# print(mask)

In [26]:
train.shape

(1004, 13)

In [27]:
test.shape

(502, 12)

## PCA

In [28]:

from sklearn.decomposition import KernelPCA
'''
pca = KernelPCA(n_components=3, kernel='rbf', gamma=.04)

# train
to_scale_train = train.drop(['ID', 'Target'], axis=1).columns


train_reduced = pd.DataFrame(pca.fit_transform(train[to_scale_train].to_numpy()))

#train[to_scale_train] = pd.DataFrame(pca.fit_transform(train[to_scale_train].to_numpy()))

# test
to_scale_test = test.drop('ID', axis=1).columns


test_reduced = pd.DataFrame(pca.fit_transform(test[to_scale_train].to_numpy()))
'''

"\npca = KernelPCA(n_components=3, kernel='rbf', gamma=.04)\n\n# train\nto_scale_train = train.drop(['ID', 'Target'], axis=1).columns\n\n\ntrain_reduced = pd.DataFrame(pca.fit_transform(train[to_scale_train].to_numpy()))\n\n#train[to_scale_train] = pd.DataFrame(pca.fit_transform(train[to_scale_train].to_numpy()))\n\n# test\nto_scale_test = test.drop('ID', axis=1).columns\n\n\ntest_reduced = pd.DataFrame(pca.fit_transform(test[to_scale_train].to_numpy()))\n"

In [29]:
#train_reduced.shape

In [30]:
#test_reduced.shape

In [31]:
#train_reduced.head()

In [32]:
#test_reduced.head()

In [33]:
#train = pd.concat([train[['ID', 'Target']], train_reduced], axis=1)
#test = pd.concat([test['ID'], test_reduced], axis=1)

In [34]:
#train.head()

In [35]:
#test.head()

##--------------------------------------------------------------------------------------------------------------

In [36]:
def process(train,test,ss) :
  
  target_mapper = dict(zip(ss.drop('ID',1).columns.tolist(),
                          [i for i in range(len(ss.drop('ID',1).columns.tolist()))]))  # Used to Encode Train Target 
  train['Target'] = train['Target'].map(target_mapper)
  
  Inversetarget_mapper = dict(zip([i for i in range(len(ss.drop('ID',1).columns.tolist()))],
                                  ss.drop('ID',1).columns.tolist())) # Used to Create submission file 

  in_cols = train.iloc[:,2:].columns.tolist() # features used in Training, we will use only time step 1 in this tutorial

  return target_mapper , Inversetarget_mapper , in_cols , train , test , ss

In [37]:
ss = pd.read_csv('SampleSubmission.csv')

In [38]:
target_mapper , Inversetarget_mapper , in_cols , train , test , ss  = process(train,test,ss)

In [39]:
#train = train.iloc[:, -24:]
#test = test.iloc[:, -24:]

In [40]:
X, y = train[in_cols], train['Target']
#X, y = train.iloc[:, -24:], train['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,stratify = y ,
                                                    random_state=58) # Random state keeps the split consistent
print(X_train.shape, X_test.shape)

(803, 11) (201, 11)


In [41]:
test.head()

,ID,train_ndvi_1,train_ndvi_3,train_ndvi_4,train_ndvi_6,train_ndvi_7,train_ndvi_8,train_ndvi_11,train_ndvi_12,train_ndvi_13,train_ndvi_14,train_ndvi_15
0,d8da32b5,0.319765,0.024700,0.014195,0.029400,0.040636,-0.142692,0.011390,-0.064257,0.078806,0.195170,0.134185
1,670ad0fb,-0.106655,-0.469677,-0.389538,-0.780399,-1.178829,-0.967963,-1.125707,-0.878354,-0.486408,0.053841,-0.458233
2,fec40ac9,0.080779,-0.723073,-0.778036,-0.129596,-0.076592,-0.395590,0.266057,1.611051,1.673907,1.333393,-0.632236
3,4f6d4495,1.547509,0.342662,-0.049763,-0.091414,0.114688,-0.092243,0.930763,0.653914,0.916741,1.155769,0.677791
4,e56d2db7,-1.653209,1.168643,0.636218,-0.022393,0.027116,-0.419941,-0.643338,-0.734600,-0.771260,-0.953040,-0.745766


In [42]:
print(y_train.shape)

(803,)


## Model

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
'''
#model = RandomForestClassifier(random_state=0) # Create the model
model = OneVsRestClassifier(LogisticRegression(random_state=0))

model.fit(X_train, y_train) # Train it (this syntax looks the same for all sklearn models)


best_params = {'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 3, 'n_estimators': 150}
model = RandomForestClassifier(random_state=0, criterion='entropy', max_features='auto', min_samples_leaf=3, n_estimators=150) # Create the model
model.fit(X_train, y_train) # Train it (this syntax looks the same for all sklearn models)



!python -m pip install catboost

from catboost import CatBoostRegressor
model = CatBoostRegressor(n_estimators=100,
                       loss_function='RMSE',
                       learning_rate=0.1,
                       depth=3, task_type='CPU',
                       random_state=1,
                       verbose=False)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

'''
# Random Forest

model = RandomForestClassifier(criterion= 'entropy', max_features= 'auto', min_samples_leaf= 3, n_estimators= 200)
model.fit(X_train, y_train)
print('LOCAL log_loss',log_loss(y_test, model.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, model.predict(X_test)))


LOCAL log_loss 1.9402381290993869
LOCAL accuracy 0.43781094527363185


In [44]:
X_train.head()

,train_ndvi_1,train_ndvi_3,train_ndvi_4,train_ndvi_6,train_ndvi_7,train_ndvi_8,train_ndvi_11,train_ndvi_12,train_ndvi_13,train_ndvi_14,train_ndvi_15
617,0.997666,1.547302,0.788885,-0.123518,-0.218843,-0.437716,-0.308225,-0.184769,-0.109632,-0.065435,1.873651
537,0.108710,-0.639399,-0.779506,-0.505030,-0.485235,-0.398578,-0.471634,-0.372639,-0.219708,-0.133578,-0.463681
583,-0.755633,-0.408452,-0.541546,-0.920282,-0.912339,-1.065341,-1.215317,-1.042387,-0.557271,-0.473729,-0.531612
44,1.372682,3.134842,3.117040,3.476270,3.186270,1.992356,0.084548,-1.528490,-1.136242,-1.470901,2.793133
670,-0.045629,-0.947247,-0.907406,-0.914205,-0.850308,-0.462129,-0.427414,-0.276078,-0.362558,-0.356609,-0.273414


In [45]:
'''
# Split the training set into a development and an evaluation sets
from sklearn.model_selection import train_test_split
X_dev, X_eval, y_dev, y_eval = train_test_split(X_train,
                                                y_train,
                                                test_size=0.2,
                                                random_state=42)

X_test.head(10)
'''

'\n# Split the training set into a development and an evaluation sets\nfrom sklearn.model_selection import train_test_split\nX_dev, X_eval, y_dev, y_eval = train_test_split(X_train,\n                                                y_train,\n                                                test_size=0.2,\n                                                random_state=42)\n\nX_test.head(10)\n'

In [46]:
'''from sklearn.model_selection import GridSearchCV

# 1. Random Forest
import time
dict_clf = {}


paramgrid = {
    'n_estimators':      [150, 300]
}
GS = GridSearchCV(RandomForestClassifier(random_state=77),
                  paramgrid,
                  cv=4)

# Fit the data and record time taking to train
t0 = time.time()
GS.fit(X_dev, y_dev)
t = time.time() - t0

# Store best parameters, score and estimator
model = GS.best_estimator_
best_params = GS.best_params_
best_score = GS.best_score_

print('LOCAL log_loss',log_loss(y_test, best_clf.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, best_clf.predict(X_test)))
print(best_score)

# name = 'RF'
'''

"from sklearn.model_selection import GridSearchCV\n\n# 1. Random Forest\nimport time\ndict_clf = {}\n\n\nparamgrid = {\n    'n_estimators':      [150, 300]\n}\nGS = GridSearchCV(RandomForestClassifier(random_state=77),\n                  paramgrid,\n                  cv=4)\n\n# Fit the data and record time taking to train\nt0 = time.time()\nGS.fit(X_dev, y_dev)\nt = time.time() - t0\n\n# Store best parameters, score and estimator\nmodel = GS.best_estimator_\nbest_params = GS.best_params_\nbest_score = GS.best_score_\n\nprint('LOCAL log_loss',log_loss(y_test, best_clf.predict_proba(X_test))) # predict_proba return probabilities \nprint('LOCAL accuracy',accuracy_score(y_test, best_clf.predict(X_test)))\nprint(best_score)\n\n# name = 'RF'\n"

In [47]:
#print(best_params)

In [48]:
print('LOCAL log_loss',log_loss(y_test, model.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, model.predict(X_test)))

LOCAL log_loss 1.9402381290993869
LOCAL accuracy 0.43781094527363185


In [49]:
test_prediction = model.predict_proba(test[in_cols])

sub = pd.DataFrame(test_prediction)
sub = sub.rename(columns=Inversetarget_mapper)
sub['ID'] =  test['ID'].values
submission = pd.read_csv('SampleSubmission.csv')
submission = pd.merge(sub[['ID']],submission,on='ID',how='left')

for col in submission.columns[1:] :
  submission[col] = sub[col]
  
submission.head()

,ID,arable_soil,cereals,forage_crop,greenhouses,mixed_crops,ochards,olive,olive+arbo,olive+cereals,olive+crops,plowing_and_sowing,vegetable_and_flower,wheat
0,d8da32b5,0.042086,0.019359,0.029080,0.000000,0.086315,0.055228,0.533177,0.183353,0.021510,0.025468,0.001548,0.000000,0.002875
1,670ad0fb,0.093139,0.038909,0.006444,0.001250,0.019470,0.069161,0.526537,0.061663,0.064716,0.049988,0.018649,0.012232,0.037841
2,fec40ac9,0.043946,0.007286,0.027540,0.006583,0.073635,0.209482,0.331169,0.065901,0.016440,0.089933,0.042173,0.082697,0.003214
3,4f6d4495,0.072250,0.086818,0.068100,0.000625,0.018530,0.167878,0.234158,0.182405,0.084030,0.057716,0.006528,0.001000,0.019962
4,e56d2db7,0.094919,0.162442,0.057163,0.010351,0.016608,0.015478,0.114326,0.012617,0.132188,0.027069,0.070451,0.001250,0.285138


In [50]:
submission.to_csv('submission7.csv',index=False)